In [55]:
import os
import pandas as pd
from IPython.display import display, display_html
import numpy as np
import matplotlib.pyplot as plt
def samples_seen_from_str(samples):
    if samples.endswith("M"):
        samples = float(samples[:-1]) * 10**6
    elif samples.endswith("B"):
        samples = float(samples[:-1]) * 10**9
    elif samples.endswith("K"):
        samples = float(samples[:-1]) * 10**3
    return samples
def human(v):
    # check closest from `scales`
    dist = np.abs(np.array(scales_numeric) - v)
    if dist.min() < 1e9:
        idx = dist.argmin()
        return scales[idx]
    else:
        if v < 10 ** 6:
            return str(v)
        elif v > 10**6 and v < 10**9:
            return (str(v/10**6)+"M").replace(".0M", "M")
        elif v > 10**9:
            return (str(v/10**9)+"B").replace(".0B", "B")
scales = [
    "1.28M",
    "3M",
    "6.4M",
    "12.8M",
    "30M",
    "64M",
    "128M",
    "300M",
    "640M",
    "1.28B",
    "3B",
]
scales_numeric = [samples_seen_from_str(s) for s in scales]

In [ ]:
df = pd.read_csv("scaling_laws/data/all_results.csv.gz")
df["samples_seen_scale_pretty"] = df.total_samples_seen.apply(lambda s:human(s))
df = df[
    ((df.pretrain_dataset=="datacomp_1b") & (df.lr_scheduler == "cosine")) | 
    ((df.pretrain_dataset=="datacomp_1b") & (df.lr_scheduler == "const")) |
    ((df.pretrain_dataset=="relaion2b-en") & (df.lr_scheduler == "cosine"))
]
model_profile = pd.read_csv("model_profile.csv").set_index("model_simple_namespace")

In [ ]:
evals = [
 ("imagenet1k", "acc1"), # ImageNet-1k zero-shot classification
 ("mscoco_captions", "image_retrieval_recall@5"), # MS-COCO Recall@5 zero-shot image retrieval (T->I)
 ("mscoco_captions", "text_retrieval_recall@5"), # MS-COCO Recall@5 zero-shot text retrieval (I->T)
 ("datacomp_classification", "acc1"), # average over 35 classification tasks from DataComp (Tab.15 from https://arxiv.org/abs/2304.14108)
 ("imagenet_distribution_shift", "acc1"), # average over ImageNet-v2, ImageNet-R, ImageNet-Sketch, ImageNet-A, ObjectNet zero-shot classification
]   
for scheduler in ("cosine", "const",):
    for pretrain_dataset in ("datacomp_1b", "relaion2b-en"):
        for ds, metric in evals:
            for eval_type in ("similarity",):
                d = df.copy()
                d = d[d.eval_type==eval_type]
                d = d[d.pretrain_dataset==pretrain_dataset]
                d = d[d.downstream_dataset==ds]                        
                if scheduler != "const":            
                    d = d[d.epoch==d.total_epochs]
                d = d[d.lr_scheduler==scheduler]
                d = d[d.samples_seen_scale_pretty.isin(scales)]
                vars=("samples_seen_scale_pretty", "model_simple_namespace")
                d = d.sort_values(by=metric, ascending=False)
                d = d.drop_duplicates(subset=vars, keep="first") # Show best result for each model x samples seen
                d = pd.pivot(
                    d,
                    index="samples_seen_scale_pretty",
                    columns=("model_simple_namespace"),
                    values=metric,
                )
                if not len(d):
                    continue
                d.index.name = "Samples Seen"
                
                d.columns.name = "Model"
                cols = d.columns
                cols = sorted(cols, key=lambda k:model_profile.loc[k].gflops)
                d = d[cols]
                d.columns = ["_".join(k.split("_")[::-1]) for k in d.columns]
                missing = [s for s in scales if s not in d.index]
                for s in missing:
                    d.loc[s] = np.nan
                index = sorted(d.index, key=lambda k:samples_seen_from_str(k))
                d = d.loc[index]
                d = d.T
                        
                maxval = np.nanmax(d.values)
                minval = np.nanmin(d.values)
                style = d.style.background_gradient(cmap="viridis", vmin=minval, vmax=maxval, axis=None)                    
                style = style.format(precision=3, na_rep="NA")
                display_html(f"<font color='green'>'{ds}'</font> {metric} results for models pre-trained on <font color='red'>'{pretrain_dataset}'</font>  using scheduler <font color='blue'>'{scheduler}'</font>:", raw=True)            
                display(style)
            

'imagenet1k' acc1 results for models pre-trained on 'datacomp_1b' using scheduler 'cosine' :

Samples Seen,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B
clip_ViT-S-32,0.013,0.030,0.060,0.102,0.187,0.270,0.354,0.440,0.497,0.531,0.559
siglip_ViT-S-32,0.013,0.029,0.056,0.096,0.177,0.262,0.346,0.428,0.487,NA,NA
clip_ViT-M-32,0.014,0.032,0.063,0.105,0.201,0.292,0.383,0.478,0.537,0.573,0.608
siglip_ViT-M-32,0.014,0.030,0.059,0.101,0.191,0.283,0.373,0.468,0.525,NA,NA
clip_ViT-S-16,0.020,0.044,0.080,0.134,0.237,0.329,0.423,0.505,NA,0.596,0.628
siglip_ViT-S-16,NA,NA,NA,NA,NA,NA,0.414,NA,NA,NA,NA
mammut_ViT-S-32,0.015,0.033,0.064,0.111,0.204,0.293,0.372,0.452,0.503,0.514,0.565
clip_ViT-S-14,0.020,0.046,0.080,0.135,0.237,NA,0.420,0.513,NA,0.598,0.637
clip_ViT-B-32,0.014,0.032,0.065,0.112,0.216,0.312,0.404,0.503,NA,0.608,0.653
siglip_ViT-B-32,0.013,0.031,0.062,0.108,0.206,0.302,0.397,0.498,0.561,NA,NA


'mscoco_captions' image_retrieval_recall@5 results for models pre-trained on 'datacomp_1b' using scheduler 'cosine' :

Samples Seen,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B
clip_ViT-S-32,0.020,0.038,0.066,0.106,0.183,0.256,0.326,0.394,0.440,0.477,0.497
siglip_ViT-S-32,0.020,0.036,0.064,0.104,0.181,0.250,0.320,0.388,0.438,NA,NA
clip_ViT-M-32,0.022,0.039,0.070,0.113,0.197,0.276,0.349,0.426,0.477,0.517,0.547
siglip_ViT-M-32,0.021,0.037,0.066,0.112,0.193,0.267,0.346,0.420,0.475,NA,NA
clip_ViT-S-16,0.029,0.053,0.087,0.138,0.228,0.308,0.390,0.457,NA,0.536,0.557
siglip_ViT-S-16,NA,NA,NA,NA,NA,NA,0.382,NA,NA,NA,NA
mammut_ViT-S-32,0.020,0.038,0.068,0.118,0.201,0.278,0.346,0.410,0.456,0.459,0.507
clip_ViT-S-14,0.032,0.054,0.086,0.145,0.234,NA,0.384,0.454,NA,0.540,0.566
clip_ViT-B-32,0.019,0.039,0.072,0.119,0.209,0.294,0.372,0.453,NA,0.548,0.586
siglip_ViT-B-32,0.021,0.037,0.067,0.114,0.204,0.283,0.365,0.445,0.503,NA,NA


'mscoco_captions' text_retrieval_recall@5 results for models pre-trained on 'datacomp_1b' using scheduler 'cosine' :

...

Samples Seen,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B
clip_ViT-S-32-alt,NA,NA,NA,0.003,0.013,0.037,0.138,0.310,0.391,0.447,0.462
clip_ViT-S-32,0.021,0.039,0.069,0.089,0.175,0.248,0.325,0.358,0.416,0.469,0.485
clip_ViT-M-32-alt,NA,NA,NA,0.003,0.015,0.049,0.180,0.364,0.448,0.502,0.521
clip_ViT-S-16-alt,NA,NA,NA,0.003,0.015,0.052,0.171,0.367,0.452,0.506,0.525
clip_ViT-M-32,0.022,0.042,0.076,0.095,0.188,0.276,0.352,0.390,0.465,0.513,0.532
clip_ViT-S-16,NA,NA,NA,0.003,0.016,0.052,0.190,0.398,0.478,0.533,0.554
mammut_ViT-S-32,0.021,0.042,0.076,0.094,0.015,0.053,0.212,0.363,0.422,0.435,NA
clip_ViT-S-14,NA,NA,NA,0.004,0.016,0.052,0.183,0.392,0.481,0.537,0.557
clip_ViT-B-32,0.023,0.044,0.080,0.101,0.208,0.299,0.376,0.422,0.507,0.552,0.572
clip_ViT-M-16-alt,NA,NA,NA,0.001,0.003,0.026,0.107,0.353,0.494,0.559,0.590


'mscoco_captions' image_retrieval_recall@5 results for models pre-trained on 'datacomp_1b' using scheduler 'const' :

Samples Seen,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B
clip_ViT-S-32-alt,NA,NA,NA,0.003,0.014,0.039,0.128,0.277,0.343,0.386,0.404
clip_ViT-S-32,0.027,0.048,0.082,0.099,0.170,0.240,0.303,0.343,0.368,0.417,0.440
clip_ViT-M-32-alt,NA,NA,NA,0.004,0.019,0.052,0.168,0.332,0.397,0.447,0.467
clip_ViT-S-16-alt,NA,NA,NA,0.004,0.022,0.052,0.157,0.332,0.406,0.453,0.473
clip_ViT-M-32,0.028,0.052,0.086,0.106,0.183,0.267,0.324,0.367,0.414,0.462,0.485
clip_ViT-S-16,NA,NA,NA,0.004,0.023,0.055,0.172,0.355,0.430,0.483,0.508
mammut_ViT-S-32,0.025,0.047,0.082,0.100,0.020,0.053,0.208,0.339,0.388,0.403,NA
clip_ViT-S-14,NA,NA,NA,0.004,0.023,0.048,0.165,0.352,0.435,0.483,0.511
clip_ViT-B-32,0.030,0.054,0.089,0.111,0.205,0.277,0.347,0.387,0.450,0.499,0.520
clip_ViT-M-16-alt,NA,NA,NA,0.001,0.004,0.028,0.104,0.317,0.451,0.512,0.533


'mscoco_captions' text_retrieval_recall@5 results for models pre-trained on 'datacomp_1b' using scheduler 'const' :

Samples Seen,1.28M,3M,6.4M,12.8M,30M,64M,128M,300M,640M,1.28B,3B
clip_ViT-S-32-alt,NA,NA,NA,0.004,0.027,0.056,0.194,0.421,0.502,0.555,0.578
clip_ViT-S-32,0.043,0.078,0.124,0.154,0.260,0.368,0.441,0.493,0.536,0.597,0.620
clip_ViT-M-32-alt,NA,NA,NA,0.006,0.029,0.077,0.250,0.473,0.564,0.631,0.651
clip_ViT-S-16-alt,NA,NA,NA,0.006,0.029,0.079,0.248,0.490,0.581,0.620,0.645
clip_ViT-M-32,0.043,0.079,0.138,0.167,0.276,0.397,0.480,0.526,0.591,0.647,0.664
clip_ViT-S-16,NA,NA,NA,0.005,0.034,0.082,0.255,0.534,0.610,0.661,0.683
mammut_ViT-S-32,0.041,0.077,0.127,0.156,0.028,0.079,0.306,0.493,0.555,0.577,NA
clip_ViT-S-14,NA,NA,NA,0.006,0.034,0.075,0.254,0.525,0.614,0.670,0.687
clip_ViT-B-32,0.041,0.080,0.138,0.170,0.302,0.416,0.503,0.550,0.623,0.674,0.702
clip_ViT-M-16-alt,NA,NA,NA,0.001,0.004,0.045,0.161,0.475,0.633,0.684,0.712
